Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean          SD       Naive SE        MCSE      ESS
alpha 154.59561950 0.277690332 0.0043906697 0.00389766496 1000
 beta   0.90447636 0.043027324 0.0006803217 0.00077720144 1000
sigma   5.10347559 0.198634795 0.0031406919 0.00314987455 1000

Quantiles:
          2.5%       25.0%       50.0%       75.0%       97.5%   
alpha 154.0640000 154.4010000 154.5910000 154.7880000 155.1311000
 beta   0.8203094   0.8756135   0.9045785   0.9329345   0.9890799
sigma   4.7503625   4.9601100   5.0955400   5.2348850   5.5120200



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Union{Missing, Float64}[154.941 0.92304 5.05005; 154.403 0.917218 5.1386; … ; 154.647 0.910332 5.32362; 154.569 0.900696 5.18625]

Union{Missing, Float64}[154.704 0.912819 4.92331; 154.513 0.957895 5.25348; … ; 154.585 0.912741 4.84724; 154.752 0.904219 5.41953]

Union{Missing, Float64}[154.714 0.894571 5.01794; 154.484 0.872909 5.21814; … ; 154.399 0.845601 5.12938; 154.558 0.933932 5.33229]

Union{Missing, Float64}[154.69 0.894254 5.33685; 154.677 0.94016 4.8523; … ; 154.381 0.876666 5.08252; 154.669 0.891217 4.96304]

Should be identical to earlier result

In [9]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean          SD       Naive SE        MCSE      ESS
alpha 154.59561950 0.277690332 0.0043906697 0.00389766496 1000
 beta   0.90447636 0.043027324 0.0006803217 0.00077720144 1000
sigma   5.10347559 0.198634795 0.0031406919 0.00314987455 1000

Quantiles:
          2.5%       25.0%       50.0%       75.0%       97.5%   
alpha 154.0640000 154.4010000 154.5910000 154.7880000 155.1311000
 beta   0.8203094   0.8756135   0.9045785   0.9329345   0.9890799
sigma   4.7503625   4.9601100   5.0955400   5.2348850   5.5120200



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*